In [1]:
import pandas as pd
import numpy as np

In [2]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


# Ejercicio 30

¿Cuál es la aplicación gratis con mayor ratio de reviews positivas? (⭐⭐)

Leo los csv usando únicamente las columnas que serán útiles para este ejercicio

Los valores de la columna "Reviews" de apps_details no coinciden en absoluto con la cantidad de reviews que tengo realmente en user_reviews.

Por ejemplo, si una app figura con 1000 reviews en el primer dataframe pero en el segundo solo tengo 40 reviews de esta, me falta un montón de información y calcular #(reviews positivas)/10000 sería erróneo ya que desconozco la información sobre la gran mayoría de la reviews de la app. Lo correcto sería calcular #(reviews positivas)/40.

Simplemente tendré en cuenta la cantidad de reviews por aplicación que tengo en user_reviews, ya que estas las conozco y sé si son negativas, neutrales o positivas.

Usaré Translated_Review para eliminar reviews que se encuentran duplicadas en ese csv (están idénticas, las considero errores).


In [5]:
apps_details = pd.read_csv('/content/drive/MyDrive/Organización de Datos/googleplaystore.csv', usecols=['App', 'Reviews', 'Type'])
user_reviews = pd.read_csv('/content/drive/MyDrive/Organización de Datos/googleplaystore_user_reviews.csv', usecols=['App', 'Translated_Review', 'Sentiment'])

Hay apps que figuran repetidas, no las tengo en cuenta ya que no aportarán nada nuevo a efectos de lo que busco (el valor numérico en Reviews no importa porque lo utilizaré solo para el pivot_table).

Además, hay apps con tipo inválido (0 o NaN), las podría eliminar en este momento, pero luego filtraré y me quedaré solo con las gratis, así que no molestarán.

In [6]:
apps_details[apps_details.App == 'Coloring book moana'] #ejemplo de repetición de una app (no necesario resolución)

,App,Reviews,Type
1,Coloring book moana,967,Free
2033,Coloring book moana,974,Free


In [7]:
apps_details.Type.unique() #tipos erroneos en App (no necesario resolución)

array(['Free', 'Paid', nan, '0'], dtype=object)

In [8]:
apps_details = apps_details.drop_duplicates(subset='App')

En el csv de user reviews, hay apps que tienen sentimientos con valor NaN, elimino estas ya que no aportan nada y no puedo inferir qué había en esas reviews (si el nombre es NaN tampoco me sería útil ese dato). Lo mismo haré con reviews duplicadas reviews duplicadas.

In [9]:
user_reviews[user_reviews.duplicated(subset=['App', 'Translated_Review'])].head(5) #ejemplo de reviews duplicadas (no necesario resolución)

,App,Translated_Review,Sentiment
7,10 Best Foods for You,NaN,NaN
15,10 Best Foods for You,NaN,NaN
78,10 Best Foods for You,Good,Positive
79,10 Best Foods for You,Good,Positive
100,10 Best Foods for You,I like eat delicious food. That's I'm cooking ...,Positive


In [10]:
user_reviews[user_reviews.Sentiment.isna()].head(5) #reviews con sentiment NaN (no necesario resolución)

,App,Translated_Review,Sentiment
2,10 Best Foods for You,NaN,NaN
7,10 Best Foods for You,NaN,NaN
15,10 Best Foods for You,NaN,NaN
102,10 Best Foods for You,NaN,NaN
107,10 Best Foods for You,NaN,NaN


In [11]:
user_reviews = user_reviews.drop_duplicates(subset=['App', 'Translated_Review'])
user_reviews = user_reviews.dropna() #cualquier cosa NaN en una review hace que no me sirva para el ejercicio

Hago un inner join entre los dataframes ya que voy a necesitar datos de ambos para este ejercicio. ¿Por qué inner?

Si me quedo con una app que está en apps_details pero no en user_reviews, no tendré review para esta y no me servirá

Si tengo el caso opuesto, tendré una review para una app de la que desconozco si es gratis o paga, por lo que tampoco me servirá.

In [12]:
free_apps_reviews_by_sentiment = pd.merge(apps_details[apps_details.Type == 'Free'], user_reviews, how='inner')[['App', 'Reviews', 'Sentiment']] #sólo apps gratis

In [13]:
free_apps_reviews_by_sentiment.head(3) #tengo apps gratis con sentiment por cada review (no necesario resolución)

,App,Reviews,Sentiment
0,Coloring book moana,967,Negative
1,Coloring book moana,967,Negative
2,Coloring book moana,967,Neutral



La columna Reviews la usaré para el pivot_table como valores. De todas formas da igual, si una aplicación tiene sentimiento (negativo, neutral o positivo) aparecera una valor, sino NaN (lleno estos con 0 ya que, en este caso, representa cuando no hay una review de un tipo)

In [14]:
reviews_by_app = free_apps_reviews_by_sentiment.pivot_table(index="App",columns="Sentiment",values="Reviews",aggfunc="count",fill_value=0)
reviews_by_app = reviews_by_app.reset_index()
reviews_by_app.columns = ['App', 'Negativas', 'Neutrales', 'Positivas']
reviews_by_app["Reviews"] = reviews_by_app.Negativas + reviews_by_app.Neutrales + reviews_by_app.Positivas
reviews_by_app["Ratio"] = reviews_by_app.Positivas/reviews_by_app.Reviews

reviews_by_app = reviews_by_app[['App','Positivas', 'Reviews', 'Ratio']] #negativas y neutrales ya no me importan

Hay muchas apps que sólo tienen reviews positivas, por lo que no hay una única app gratis con ratio de 1. Hay 32 que tienen este ratio.

Si hubiera considerado la cantidad de reviews del csv de apps_details, probablemente no hubiera sucedido esto pero me encontraba con el problema que mencioné en un principio, así que terminaría teniendo ratios cercanos a cero, además de que con eso no estaría usando la información de las reviews negativas o neutrales, simplemente contaría las positivas y las compararía con el valor de reviews.

In [15]:
reviews_by_app.nlargest(33, "Ratio") #recién la 33 no tiene ratio 1 (no necesario resolución)

,App,Positivas,Reviews,Ratio
12,3D Live Neon Weed Launcher,2,2,1.000000
16,7 Day Food Journal Challenge,9,9,1.000000
88,All-in-One Mahjong 3 FREE,1,1,1.000000
95,Amazon Prime Video,2,2,1.000000
118,Apartment Decorating Ideas,1,1,1.000000
167,Baby Panda’s Juice Shop,3,3,1.000000
199,Bed Time Fan - White Noise Sleep Sounds,1,1,1.000000
201,Best Fiends - Free Puzzle Game,1,1,1.000000
202,Best Ovulation Tracker Fertility Calendar App ...,1,1,1.000000
243,Brightest Flashlight Free ®,5,5,1.000000


Entonces, tenemos un conjunto de apps con mejor ratio de reviews positivas, que es el siguiente:

In [16]:
reviews_by_app[reviews_by_app.Ratio == reviews_by_app.Ratio.max()]

,App,Positivas,Reviews,Ratio
12,3D Live Neon Weed Launcher,2,2,1.0
16,7 Day Food Journal Challenge,9,9,1.0
88,All-in-One Mahjong 3 FREE,1,1,1.0
95,Amazon Prime Video,2,2,1.0
118,Apartment Decorating Ideas,1,1,1.0
167,Baby Panda’s Juice Shop,3,3,1.0
199,Bed Time Fan - White Noise Sleep Sounds,1,1,1.0
201,Best Fiends - Free Puzzle Game,1,1,1.0
202,Best Ovulation Tracker Fertility Calendar App ...,1,1,1.0
243,Brightest Flashlight Free ®,5,5,1.0


Como comentario:

Podríamos considerar que la de "mejor ratio" es la que más reviews tiene de las de ratio 1.

In [17]:
max_ratio_apps = reviews_by_app[reviews_by_app.Ratio == reviews_by_app.Ratio.max()]
max_ratio_apps[max_ratio_apps.Reviews == max_ratio_apps.Reviews.max()]

#no es necesario!!!

,App,Positivas,Reviews,Ratio
443,Down Dog: Great Yoga Anywhere,40,40,1.0
